In [ ]:
# Compatibility shim for renamed module
import sys
import app.services.ocr.paddle_ocr_service as _paddle_ocr_service
sys.modules['app.services.ocr.paddle_ocr'] = _paddle_ocr_service


## 멀티스레드 동시성 방어 테스트 개요

다음 시나리오의 테스트 코드를 작성해줘.
 1.클라이언트 에서 여러개 파일에 대해 요청이 들어온다
 2. 빠른 응답을 위해 여러개 파일에 대해 병렬로 ImagePreprocessor.process_bytes + PaddleOCRService.run_ocr_from_bytes + LabTableExtractor.extract 를 실행한다.
 3. 병렬 실행이 모두 완료되면 실행된 결과들을 하나의 리스트로 병합한다, 단 리스트 element 의 순서는 요청된 파일의 순서에 따른다.
 4. 최종 결과를 pretty json 으로 출력한다.

In [4]:
# 병렬 OCR 파이프라인 실행: ImagePreprocessor.process_bytes → PaddleOCRService.run_ocr_from_bytes → (lines) → LabTableExtractor.extract
import os
import json
import threading
import uuid
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from collections import defaultdict

# 레포 내 예시 이미지 경로(요청 순서를 그대로 유지)
img_paths = [
    os.path.abspath('notebooks/ocr/assets/images/20241106.jpg'),
    os.path.abspath('notebooks/ocr/assets/images/20241107_2.jpg'),
    os.path.abspath('notebooks/ocr/assets/images/20241107.jpg'),
]

# 서비스 임포트
from app.services.analysis.image_preprocessor import ImagePreprocessor
from app.services.ocr.paddle_ocr import PaddleOCRService
from app.services.analysis.lab_table_extractor import LabTableExtractor
from app.services.analysis import line_preprocessor as lp

# 인스턴스 준비(필요 시 환경변수로 설정 제어 가능)
image_prep = ImagePreprocessor()
ocr_service = PaddleOCRService()

# LabTableExtractor 초기화: 프로젝트 상태에 따라 인자 시그니처가 다를 수 있어 방어적으로 처리
try:
    extractor = LabTableExtractor()
except TypeError:
    # 예: 내부에서 OpenAI api_key가 필요한 경우를 대비
    extractor = LabTableExtractor(api_key=os.getenv("OPENAI_API_KEY"))

RUN_ID = uuid.uuid4().hex[:8]
_start_counts = defaultdict(int)
_counts_lock = threading.Lock()


def _ts():
    # 밀리초 단위 타임스탬프 문자열
    return datetime.now().strftime('%H:%M:%S.%f')[:-3]

# 단일 파일 처리 함수(예외 발생 시 에러 정보 포함 반환)
def process_one(index: int, path: str):
    basename = os.path.basename(path)
    thr = threading.current_thread().name
    with _counts_lock:
        _start_counts[index] += 1
        c = _start_counts[index]
    dup = " (중복호출)" if c > 1 else ""
    print(f"[{_ts()}][{RUN_ID}] 시작: {basename} (idx={index}, thr={thr})#{c}{dup}", flush=True)
    try:
        with open(path, "rb") as f:
            raw_bytes = f.read()
        # 1) 전처리
        pre_bytes = image_prep.process_bytes(raw_bytes)
        # 2) OCR (원본 결과는 보통 페이지별 dict 리스트)
        print(f"[{_ts()}][{RUN_ID}]  OCR 호출: {basename} (thr={thr})", flush=True)
        ocr_result = ocr_service.run_ocr_from_bytes(pre_bytes)
        if ocr_result is None:
            print(f"[{_ts()}][{RUN_ID}]  OCR 실패: {basename} (thr={thr})", flush=True)
            return (index, {"ok": False, "error": "OCR returned None", "path": path, "run_id": RUN_ID})
        # 3) 라인 그룹핑 입력을 1개 dict로 정규화
        if isinstance(ocr_result, list):
            if len(ocr_result) == 0:
                print(f"[{_ts()}][{RUN_ID}]  빈 OCR 결과: {basename} (thr={thr})", flush=True)
                return (index, {"ok": True, "data": extractor.extract([]), "run_id": RUN_ID})
            ocr_page = ocr_result[0]
        else:
            ocr_page = ocr_result
        # 4) OCR → 라인 배열로 변환
        lines = lp.extract_and_group_lines(ocr_page, isDebug=False)
        # 5) 최종 스키마 형태 결과
        final_json = extractor.extract(lines)
        print(f"[{_ts()}][{RUN_ID}] 완료: {basename} (idx={index}, thr={thr})", flush=True)
        return (index, {"ok": True, "data": final_json, "run_id": RUN_ID})
    except Exception as e:
        print(f"[{_ts()}][{RUN_ID}] 오류: {basename} → {e} (thr={thr})", flush=True)
        return (index, {"ok": False, "error": str(e), "path": path, "run_id": RUN_ID})

# 병렬 실행(요청 순서를 유지하도록 index를 함께 전달)
indexed_paths = list(enumerate(img_paths))
max_workers = min(4, len(indexed_paths)) or 1
print(f"[{_ts()}][{RUN_ID}] 병렬 실행 시작: 작업 {len(indexed_paths)}개, 워커 {max_workers}개", flush=True)

results = []
with ThreadPoolExecutor(max_workers=max_workers, thread_name_prefix=f"pool-{RUN_ID}") as ex:
    futures = {ex.submit(process_one, idx, path): (idx, path) for idx, path in indexed_paths}
    for fut in as_completed(futures):
        results.append(fut.result())

print(f"[{_ts()}][{RUN_ID}] 병렬 실행 종료", flush=True)

# 입력 순서(index) 기준으로 정렬 후, 결과 리스트 구성
results.sort(key=lambda x: x[0])
final_results = [item[1] for item in results]

# 각 index의 호출 횟수 요약(디버그)
with _counts_lock:
    call_summary = {str(k): v for k, v in sorted(_start_counts.items())}

# Pretty JSON 출력
print(json.dumps({
  "run_id": RUN_ID,
  "call_summary": call_summary,
  "results": final_results
}, ensure_ascii=False, indent=2))

Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/PP-LCNet_x1_0_doc_ori`.
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/PP-LCNet_x1_0_doc_ori`.
Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/UVDoc`.
Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/UVDoc`.
Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/PP-LCNet_x1_0_textline_ori`.
Creating model: ('PP-LCNet_x1_0_textline_ori', None)

[03:27:56.819][1871c9d9] 병렬 실행 시작: 작업 3개, 워커 3개
[03:27:56.820][1871c9d9] 시작: 20241106.jpg (idx=0, thr=pool-1871c9d9_0)#1
[03:27:56.820][1871c9d9] 시작: 20241107_2.jpg (idx=1, thr=pool-1871c9d9_1)#1
[03:27:56.823][1871c9d9] 시작: 20241107.jpg (idx=2, thr=pool-1871c9d9_2)#1
[03:27:56.820][1871c9d9] 시작: 20241106.jpg (idx=0, thr=pool-1871c9d9_0)#1
[03:27:56.820][1871c9d9] 시작: 20241107_2.jpg (idx=1, thr=pool-1871c9d9_1)#1
[03:27:56.823][1871c9d9] 시작: 20241107.jpg (idx=2, thr=pool-1871c9d9_2)#1
[03:27:57.682][1871c9d9]  OCR 호출: 20241107_2.jpg (thr=pool-1871c9d9_1)
[03:27:57.682][1871c9d9]  OCR 호출: 20241107_2.jpg (thr=pool-1871c9d9_1)
[03:27:57.833][1871c9d9]  OCR 호출: 20241107.jpg (thr=pool-1871c9d9_2)
[03:27:57.833][1871c9d9]  OCR 호출: 20241107.jpg (thr=pool-1871c9d9_2)
[03:27:57.875][1871c9d9]  OCR 호출: 20241106.jpg (thr=pool-1871c9d9_0)
[03:27:57.875][1871c9d9]  OCR 호출: 20241106.jpg (thr=pool-1871c9d9_0)
[03:28:10.003][1871c9d9] 완료: 20241107_2.jpg (idx=1, thr=pool-1871c9d9_1)
[03:28:10.003][1871